In [1]:
import logging
import json
from datetime import datetime
import pandas as pd
from autoIG.utils import close_open_positions

import numpy as np
from trading_ig.rest import IGService
from trading_ig.config import config
# import yfinance as yf
import matplotlib.pyplot as plt
from autoIG.notebook_utils import market_series
from autoIG.config import ig_service_config


In [2]:
## Instatiate IG Service
ig_service = IGService(**ig_service_config)
ig = ig_service.create_session()
# ig

In [29]:
## Search for a market
results = ig_service.search_markets('crypto')
results.head(2)

,epic,instrumentName,instrumentType,expiry,high,low,percentageChange,netChange,updateTime,updateTimeUTC,bid,offer,delayTime,streamingPricesAvailable,marketStatus,scalingFactor
0,CS.D.CRYPTOB10.TODAY.IP,Crypto 10 Index,CURRENCIES,DFB,6156.1,5985.2,0.18,10.73,14:10:42,14:10:42,6009.59,6129.59,0,True,TRADEABLE,1
1,CS.D.BITCOIN.TODAY.IP,Bitcoin,CURRENCIES,DFB,17243.2,17058.1,0.08,13.10,14:10:43,14:10:43,17104.00,17194.00,0,True,TRADEABLE,1


In [30]:
## Get info about a market
from autoIG.instruments import Epics
market = ig_service.fetch_market_by_epic(Epics.GOLD_EPIC.value)
print(f"Keys: {list(market.keys())}")

Keys: ['instrument', 'dealingRules', 'snapshot']


In [31]:
## Details about the market instrument
i,d,s = market_series(market)

## Get historical prices

In [8]:
result = ig_service.fetch_historical_prices_by_epic(epic = US_CRUDE_OIL_EPIC,resolution ='1Min',numpoints = 10 )
print(f"Keys: {list(result.keys())}")

Keys: ['prices', 'instrumentType', 'metadata']


In [9]:
# ask price is the lowest a seller will sell
# bid is highest a buyer will buy
# You buy at bid. sell at ask
result['prices'].head(2)

bid                             ask                  \
                       Open    High     Low   Close    Open    High     Low   
DateTime                                                                      
2022-12-06 20:14:00  7439.2  7440.2  7437.2  7438.7  7442.0  7443.3  7440.3   
2022-12-06 20:15:00  7438.5  7440.8  7437.5  7438.5  7442.3  7444.3  7441.0   

                            last                        
                      Close Open High Low Close Volume  
DateTime                                                
2022-12-06 20:14:00  7442.5  NaN  NaN NaN   NaN     44  
2022-12-06 20:15:00  7441.3  NaN  NaN NaN   NaN     78

In [10]:
result['instrumentType']

'COMMODITIES'

In [11]:
result['metadata']

{'allowance': {'remainingAllowance': 9867,
  'totalAllowance': 10000,
  'allowanceExpiry': 598702},
 'size': 10,
 'pageData': {'pageSize': 20, 'pageNumber': 1, 'totalPages': 1}}

In [25]:
## open a possition
from autoIG.epics import BITCOIN_EPIC,GOLD_EPIC,US_CRUDE_OIL_EPIC
resp = ig_service.create_open_position(
        currency_code='GBP',
        direction='BUY',
        epic=US_CRUDE_OIL_EPIC,
        order_type='MARKET',
        expiry='DFB',
        force_open='false',
        guaranteed_stop='false',
        size=1.0,
        level=None,
        limit_distance=None,
        limit_level=None,
        quote_id=None,
        stop_level=None,
        stop_distance=None,
        trailing_stop=None,
        trailing_stop_increment=None
        )
resp

{'date': '2022-12-08T13:53:20.108',
 'status': 'OPEN',
 'reason': 'SUCCESS',
 'dealStatus': 'ACCEPTED',
 'epic': 'CC.D.CL.USS.IP',
 'expiry': 'DFB',
 'dealReference': 'ZFPZ72ZMNSTZEB',
 'dealId': 'DIAAAALBUJ7XZA6',
 'affectedDeals': [{'dealId': 'DIAAAALBUJ7XZA6', 'status': 'OPENED'}],
 'level': 7483.6,
 'size': 1.0,
 'direction': 'BUY',
 'stopLevel': None,
 'limitLevel': None,
 'stopDistance': None,
 'limitDistance': None,
 'guaranteedStop': False,
 'trailingStop': False,
 'profit': None,
 'profitCurrency': None}

In [26]:
resp['dealStatus']

'ACCEPTED'

In [3]:
open_positions = ig_service.fetch_open_positions()
open_positions

,contractSize,createdDate,createdDateUTC,dealId,dealReference,size,direction,limitLevel,level,currency,...,percentageChange,netChange,bid,offer,updateTime,updateTimeUTC,delayTime,streamingPricesAvailable,marketStatus,scalingFactor
0,1.0,2023/01/18 18:43:30:000,2023-01-18T18:43:30,DIAAAALLQVQSAA4,MK99UTKJX36TZEB,1.0,BUY,None,8047.7,GBP,...,-0.35,-28.0,7945.0,7948.8,21:59:58,21:59:58,0,True,EDITS_ONLY,1
1,1.0,2023/01/18 18:50:00:000,2023-01-18T18:50:00,DIAAAALLQV3DMAP,NL5M2RM2UDATZEB,1.0,BUY,None,8045.7,GBP,...,-0.35,-28.0,7945.0,7948.8,21:59:58,21:59:58,0,True,EDITS_ONLY,1
2,1.0,2023/01/27 17:00:00:000,2023-01-27T17:00:00,DIAAAALNHRMHPAY,7Y4DB7JENL6TZEB,1.0,BUY,None,8017.9,GBP,...,-0.35,-28.0,7945.0,7948.8,21:59:58,21:59:58,0,True,EDITS_ONLY,1
3,1.0,2023/01/27 17:21:00:000,2023-01-27T17:21:00,DIAAAALNHR77VAR,3SYNBLHFU26TZEB,1.0,BUY,None,7985.9,GBP,...,-0.35,-28.0,7945.0,7948.8,21:59:58,21:59:58,0,True,EDITS_ONLY,1
4,1.0,2023/01/27 17:21:45:000,2023-01-27T17:21:45,DIAAAALNHSUHEAZ,HX4LB3U37LATZEB,1.0,BUY,None,7989.9,GBP,...,-0.35,-28.0,7945.0,7948.8,21:59:58,21:59:58,0,True,EDITS_ONLY,1
5,1.0,2023/01/27 17:37:00:000,2023-01-27T17:37:00,DIAAAALNHS9DPAM,G5W3LVEWXPSTZEB,1.0,BUY,None,7929.9,GBP,...,-0.35,-28.0,7945.0,7948.8,21:59:58,21:59:58,0,True,EDITS_ONLY,1


In [10]:
close_open_positions(open_positions.dealId,ig_service)


In [9]:
logging.basicConfig(format='%(asctime)s %(levelname)-8s %(module)-20s %(message)s',level=logging.INFO,datefmt='%Y-%m-%d %H:%M:%S',)

In [11]:
open_positions = ig_service.fetch_open_positions()
open_positions

,contractSize,createdDate,createdDateUTC,dealId,dealReference,size,direction,limitLevel,level,currency,...,percentageChange,netChange,bid,offer,updateTime,updateTimeUTC,delayTime,streamingPricesAvailable,marketStatus,scalingFactor
0,1.0,2023/01/18 18:43:30:000,2023-01-18T18:43:30,DIAAAALLQVQSAA4,MK99UTKJX36TZEB,1.0,BUY,None,8047.7,GBP,...,-0.35,-28.0,7945.0,7948.8,21:59:58,21:59:58,0,True,EDITS_ONLY,1
1,1.0,2023/01/18 18:50:00:000,2023-01-18T18:50:00,DIAAAALLQV3DMAP,NL5M2RM2UDATZEB,1.0,BUY,None,8045.7,GBP,...,-0.35,-28.0,7945.0,7948.8,21:59:58,21:59:58,0,True,EDITS_ONLY,1
2,1.0,2023/01/27 17:00:00:000,2023-01-27T17:00:00,DIAAAALNHRMHPAY,7Y4DB7JENL6TZEB,1.0,BUY,None,8017.9,GBP,...,-0.35,-28.0,7945.0,7948.8,21:59:58,21:59:58,0,True,EDITS_ONLY,1
3,1.0,2023/01/27 17:21:00:000,2023-01-27T17:21:00,DIAAAALNHR77VAR,3SYNBLHFU26TZEB,1.0,BUY,None,7985.9,GBP,...,-0.35,-28.0,7945.0,7948.8,21:59:58,21:59:58,0,True,EDITS_ONLY,1
4,1.0,2023/01/27 17:21:45:000,2023-01-27T17:21:45,DIAAAALNHSUHEAZ,HX4LB3U37LATZEB,1.0,BUY,None,7989.9,GBP,...,-0.35,-28.0,7945.0,7948.8,21:59:58,21:59:58,0,True,EDITS_ONLY,1
5,1.0,2023/01/27 17:37:00:000,2023-01-27T17:37:00,DIAAAALNHS9DPAM,G5W3LVEWXPSTZEB,1.0,BUY,None,7929.9,GBP,...,-0.35,-28.0,7945.0,7948.8,21:59:58,21:59:58,0,True,EDITS_ONLY,1


## Account activity

In [33]:
from_date = datetime(2021, 1, 1)
activities = ig_service.fetch_account_activity(from_date=from_date)
activities.head(3)

,date,epic,period,dealId,channel,type,status,description,details
0,2022-12-11T14:02:00,CS.D.BITCOIN.TODAY.IP,DFB,DIAAAALB5TRMZAP,PUBLIC_WEB_API,POSITION,REJECTED,Rejected: CLOSING ORDERS ONLY,None
1,2022-12-11T14:01:00,CS.D.BITCOIN.TODAY.IP,DFB,DIAAAALB5TVJ7A8,PUBLIC_WEB_API,POSITION,REJECTED,Rejected: CLOSING ORDERS ONLY,None
2,2022-12-11T14:00:01,CS.D.BITCOIN.TODAY.IP,DFB,DIAAAALB5TNTVAY,PUBLIC_WEB_API,POSITION,REJECTED,Rejected: CLOSING ORDERS ONLY,None


In [35]:
activities

,date,epic,period,dealId,channel,type,status,description,details
0,2022-12-11T14:02:00,CS.D.BITCOIN.TODAY.IP,DFB,DIAAAALB5TRMZAP,PUBLIC_WEB_API,POSITION,REJECTED,Rejected: CLOSING ORDERS ONLY,None
1,2022-12-11T14:01:00,CS.D.BITCOIN.TODAY.IP,DFB,DIAAAALB5TVJ7A8,PUBLIC_WEB_API,POSITION,REJECTED,Rejected: CLOSING ORDERS ONLY,None
2,2022-12-11T14:00:01,CS.D.BITCOIN.TODAY.IP,DFB,DIAAAALB5TNTVAY,PUBLIC_WEB_API,POSITION,REJECTED,Rejected: CLOSING ORDERS ONLY,None
3,2022-12-11T13:59:00,CS.D.BITCOIN.TODAY.IP,DFB,DIAAAALB5TE9EA3,PUBLIC_WEB_API,POSITION,REJECTED,Rejected: CLOSING ORDERS ONLY,None
4,2022-12-10T15:43:56,CC.D.CL.USS.IP,DFB,DIAAAALB5MVLKAZ,PUBLIC_WEB_API,POSITION,REJECTED,Rejected: Market closed,None
...,...,...,...,...,...,...,...,...,...
414,2021-10-25T13:46:55,UB.D.DRVNUS.DAILY.IP,DFB,DIAAAAGUFKR9DAV,WEB,WORKING_ORDER,REJECTED,Rejected: Order level too close to market level,None
415,2021-10-21T13:09:43,CS.D.GBPUSD.TODAY.IP,DFB,DIAAAAGTVTNWYA2,WEB,POSITION,ACCEPTED,Position/s closed: TVTLBXAJ,None
416,2021-10-21T13:08:46,CS.D.GBPUSD.TODAY.IP,DFB,DIAAAAGTVTLBXAJ,WEB,POSITION,ACCEPTED,Position opened: TVTLBXAJ,None
417,2021-10-15T13:37:21,CS.D.USDJPY.TODAY.IP,DFB,DIAAAAGR94EW6BD,WEB,POSITION,ACCEPTED,Position/s closed: R93HRKAV,None


In [96]:
import requests
from bs4 import BeautifulSoup
import re
import json
import base64
url= 'https://app.powerbi.com/view?r=eyJrIjoiMjA1ODM0ZWEtMmMyMS00ZDVjLTg1MzctODFkZDVhZDVmZGNlIiwidCI6ImExMmNlNTRiLTNkM2QtNDM0Ni05NWVmLWZmMTNjYTVkZDQ3ZCJ9'
soup = BeautifulSoup(requests.get(url).content, 'html.parser')

In [102]:
soup


<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>Microsoft Power BI</title>
<meta content="IE=edge;" http-equiv="X-UA-Compatible"/>
<base href="/"/>
<meta charset="utf-8"/>
<meta content="" name="description"/>
<meta content="telephone=no" name="format-detection">
<link href="/images/PowerBI_Favicon.ico" rel="shortcut icon"/>
<script>
        var viewLoadingStarted = Date.now();



        var skipCheckPowerBIAccessToken = true;
        var powerBIAccessToken = 'any';
        var reportId = 'any';
        var getExplorationUrl = '/public/';
        var getConceptualSchemaUrl = '/public/reports/conceptualschema';
        var queryDataUrl = '/public/reports/querydata';
        var resourceLoaderUrl = '/public/reports/resourcePackage/';
        var routingUrl = '/public/routing/cluster/';
        var resourceLoaderIncludePackageId = true;
        var isLoadReportMessagePending = false;
        var isReportControllerInitialized = false;
        var isAnonymousEmb

In [7]:
html_data = requests.get(soup.iframe['src']).text

In [5]:
d = json.loads(base64.b64decode(soup.iframe['src'].split('=')[-1]).decode('utf-8'))


266